# Diffusion Model Customization

**對應課程**: 李宏毅 2025 Fall GenAI-ML HW9 - Diffusion Model and Its Customization

本 notebook 介紹如何客製化預訓練的 Diffusion Model，包括：
- **Textual Inversion**: 學習新概念的文字嵌入
- **DreamBooth**: 微調整個模型學習特定主題
- **LoRA for Diffusion**: 低秩適應於擴散模型
- **ControlNet**: 增加空間控制條件

```
Diffusion 客製化方法比較：

方法              訓練參數    VRAM需求    訓練時間    效果
─────────────────────────────────────────────────────────
Textual Inversion  ~768      ~8GB        ~1hr       學習概念
DreamBooth         ~1B       ~24GB       ~30min     高品質
LoRA               ~4M       ~12GB       ~1hr       靈活高效
ControlNet         ~361M     ~16GB       ~數天      空間控制
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List, Dict, Tuple
from dataclasses import dataclass
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 1: Textual Inversion

Textual Inversion 透過學習新的文字嵌入來表示新概念，不修改模型參數。

```
Textual Inversion 流程：

訓練階段：
┌─────────────────────────────────────────────────────┐
│  "A photo of S*"     S* = 可學習的嵌入向量          │
│         │                                           │
│         ▼                                           │
│  ┌─────────────┐                                    │
│  │ Text Encoder│ ──► 文字條件 c                     │
│  └─────────────┘                                    │
│         │                                           │
│         ▼                                           │
│  ┌─────────────┐     ┌──────────┐                   │
│  │    UNet     │ ◄── │ 目標圖片 │ (3-5張)           │
│  │  (frozen)   │     └──────────┘                   │
│  └─────────────┘                                    │
│         │                                           │
│         ▼                                           │
│    更新 S* 最小化重建損失                            │
└─────────────────────────────────────────────────────┘

推理階段：
"A S* in the style of Van Gogh" → 生成該概念的新圖片
```

In [ ]:
class TextualInversionEmbedding(nn.Module):
    """Textual Inversion：學習新概念的嵌入向量"""
    
    def __init__(
        self,
        num_vectors: int = 1,  # 使用多少個 token 表示概念
        embedding_dim: int = 768,  # CLIP text encoder 維度
        initializer_token: str = "object"  # 初始化用的 token
    ):
        super().__init__()
        self.num_vectors = num_vectors
        self.embedding_dim = embedding_dim
        
        # 可學習的嵌入向量
        # 通常初始化為相似概念的嵌入
        self.learned_embedding = nn.Parameter(
            torch.randn(num_vectors, embedding_dim) * 0.02
        )
        
    def forward(self) -> torch.Tensor:
        """返回學習到的嵌入"""
        return self.learned_embedding
    
    def get_embedding_for_prompt(self, placeholder_positions: List[int], 
                                  text_embeddings: torch.Tensor) -> torch.Tensor:
        """
        將學習到的嵌入插入到文字嵌入中
        
        Args:
            placeholder_positions: placeholder token 的位置
            text_embeddings: 原始文字嵌入 [batch, seq_len, dim]
        """
        batch_size = text_embeddings.shape[0]
        modified_embeddings = text_embeddings.clone()
        
        for i, pos in enumerate(placeholder_positions):
            if i < self.num_vectors:
                # 替換 placeholder 位置的嵌入
                modified_embeddings[:, pos, :] = self.learned_embedding[i].unsqueeze(0)
                
        return modified_embeddings


# 示範
ti_embedding = TextualInversionEmbedding(num_vectors=2, embedding_dim=768)
print(f"學習參數數量: {sum(p.numel() for p in ti_embedding.parameters())}")
print(f"嵌入形狀: {ti_embedding().shape}")

In [ ]:
class TextualInversionTrainer:
    """Textual Inversion 訓練器"""
    
    def __init__(
        self,
        embedding: TextualInversionEmbedding,
        learning_rate: float = 5e-4,
        max_grad_norm: float = 1.0
    ):
        self.embedding = embedding
        self.optimizer = torch.optim.AdamW(
            embedding.parameters(),
            lr=learning_rate,
            weight_decay=1e-2
        )
        self.max_grad_norm = max_grad_norm
        
    def training_step(
        self,
        noise_pred: torch.Tensor,
        noise_target: torch.Tensor
    ) -> float:
        """
        單步訓練
        
        Args:
            noise_pred: UNet 預測的噪聲
            noise_target: 實際添加的噪聲
        """
        self.optimizer.zero_grad()
        
        # MSE 損失
        loss = F.mse_loss(noise_pred, noise_target)
        
        loss.backward()
        
        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(
            self.embedding.parameters(), 
            self.max_grad_norm
        )
        
        self.optimizer.step()
        
        return loss.item()


print("Textual Inversion 訓練配置：")
print("- 典型學習率: 5e-4 到 1e-3")
print("- 訓練步數: 3000-5000 步")
print("- 訓練圖片: 3-5 張高品質圖片")
print("- 記憶體需求: ~8GB VRAM")

## Part 2: DreamBooth

DreamBooth 透過微調整個 UNet 來學習特定主題，並使用 prior preservation 防止過擬合。

```
DreamBooth 損失函數：

L_total = L_reconstruction + λ · L_prior

L_reconstruction: 在目標圖片上的重建損失
L_prior: 在類別圖片上的先驗保留損失

┌─────────────────────────────────────────────────────────┐
│                    DreamBooth 流程                       │
├─────────────────────────────────────────────────────────┤
│                                                          │
│  輸入: 3-5 張 [V] dog 的圖片                             │
│                                                          │
│  Prompt 格式:                                            │
│  - 主題 prompt: "a [V] dog"                             │
│  - 類別 prompt: "a dog" (用於 prior preservation)       │
│                                                          │
│  訓練:                                                   │
│  ┌────────────┐   ┌────────────┐                        │
│  │ 目標圖片   │   │ 生成類別圖 │                        │
│  │ + 噪聲     │   │ + 噪聲     │                        │
│  └─────┬──────┘   └─────┬──────┘                        │
│        │                │                               │
│        ▼                ▼                               │
│     L_recon    +     L_prior                            │
│        └────────┬───────┘                               │
│                 ▼                                        │
│            更新 UNet                                     │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
@dataclass
class DreamBoothConfig:
    """DreamBooth 訓練配置"""
    # 識別符
    instance_prompt: str = "a photo of sks dog"  # [V] = "sks"
    class_prompt: str = "a photo of dog"
    
    # 訓練參數
    learning_rate: float = 5e-6  # 很小的學習率
    max_train_steps: int = 800
    train_batch_size: int = 1
    gradient_accumulation_steps: int = 1
    
    # Prior preservation
    with_prior_preservation: bool = True
    prior_loss_weight: float = 1.0
    num_class_images: int = 200  # 生成的類別圖片數量
    
    # 其他
    mixed_precision: str = "fp16"
    gradient_checkpointing: bool = True  # 節省記憶體


class DreamBoothLoss(nn.Module):
    """DreamBooth 損失函數"""
    
    def __init__(self, prior_loss_weight: float = 1.0):
        super().__init__()
        self.prior_loss_weight = prior_loss_weight
        
    def forward(
        self,
        model_pred_instance: torch.Tensor,
        target_instance: torch.Tensor,
        model_pred_class: Optional[torch.Tensor] = None,
        target_class: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, Dict[str, float]]:
        """
        計算 DreamBooth 損失
        
        Args:
            model_pred_instance: 在實例圖片上的預測
            target_instance: 實例圖片的目標噪聲
            model_pred_class: 在類別圖片上的預測 (prior preservation)
            target_class: 類別圖片的目標噪聲
        """
        # 實例重建損失
        instance_loss = F.mse_loss(model_pred_instance, target_instance)
        
        metrics = {'instance_loss': instance_loss.item()}
        
        # Prior preservation 損失
        if model_pred_class is not None and target_class is not None:
            prior_loss = F.mse_loss(model_pred_class, target_class)
            total_loss = instance_loss + self.prior_loss_weight * prior_loss
            metrics['prior_loss'] = prior_loss.item()
        else:
            total_loss = instance_loss
            
        metrics['total_loss'] = total_loss.item()
        
        return total_loss, metrics


# 示範
config = DreamBoothConfig()
print(f"DreamBooth 配置:")
print(f"  Instance prompt: {config.instance_prompt}")
print(f"  Class prompt: {config.class_prompt}")
print(f"  Learning rate: {config.learning_rate}")
print(f"  Prior preservation: {config.with_prior_preservation}")

In [ ]:
def visualize_dreambooth_concept():
    """視覺化 DreamBooth 概念"""
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    
    # 模擬訓練圖片
    np.random.seed(42)
    for i, ax in enumerate(axes[0]):
        # 創建假的狗圖片（用顏色塊代表）
        img = np.ones((64, 64, 3)) * 0.8
        # 添加一些特徵
        img[20:45, 20:45] = [0.6, 0.4, 0.2]  # 棕色主體
        img[25:30, 25:30] = [0.1, 0.1, 0.1]  # 眼睛
        img[25:30, 35:40] = [0.1, 0.1, 0.1]  # 眼睛
        img[32:35, 30:35] = [0.2, 0.1, 0.1]  # 鼻子
        # 添加一些變化
        img += np.random.randn(64, 64, 3) * 0.05
        img = np.clip(img, 0, 1)
        
        ax.imshow(img)
        ax.set_title(f"訓練圖片 {i+1}\n'sks dog'")
        ax.axis('off')
    
    # 模擬生成圖片
    prompts = [
        "sks dog on beach",
        "sks dog painting",
        "sks dog in snow"
    ]
    colors = [[0.9, 0.8, 0.6], [0.7, 0.5, 0.8], [0.95, 0.95, 1.0]]
    
    for i, (ax, prompt, bg_color) in enumerate(zip(axes[1], prompts, colors)):
        img = np.ones((64, 64, 3)) * np.array(bg_color)
        # 保持相同的狗特徵
        img[20:45, 20:45] = [0.6, 0.4, 0.2]
        img[25:30, 25:30] = [0.1, 0.1, 0.1]
        img[25:30, 35:40] = [0.1, 0.1, 0.1]
        img[32:35, 30:35] = [0.2, 0.1, 0.1]
        img += np.random.randn(64, 64, 3) * 0.03
        img = np.clip(img, 0, 1)
        
        ax.imshow(img)
        ax.set_title(f"生成結果\n'{prompt}'")
        ax.axis('off')
    
    plt.suptitle("DreamBooth: 用少量圖片學習特定主題", fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_dreambooth_concept()

## Part 3: LoRA for Stable Diffusion

LoRA (Low-Rank Adaptation) 可以高效地客製化 Diffusion Model，只訓練低秩分解的權重。

```
LoRA 在 Stable Diffusion 中的應用：

原始權重 W ∈ R^(d×k)
LoRA: W' = W + BA，其中 B ∈ R^(d×r), A ∈ R^(r×k), r << min(d,k)

應用位置：
┌─────────────────────────────────────────────────────┐
│                    UNet 架構                         │
├─────────────────────────────────────────────────────┤
│                                                      │
│  Cross-Attention 層 (最重要):                       │
│  ├─ to_q: Query 投影 ← LoRA                        │
│  ├─ to_k: Key 投影   ← LoRA                        │
│  ├─ to_v: Value 投影 ← LoRA                        │
│  └─ to_out: 輸出投影 ← LoRA                        │
│                                                      │
│  Self-Attention 層 (可選):                          │
│  └─ 同上結構                                        │
│                                                      │
│  Rank 選擇:                                          │
│  ├─ r=4:  最小，適合簡單風格                        │
│  ├─ r=8:  平衡，常用選擇                            │
│  ├─ r=16: 較強表達力                                │
│  └─ r=64: 接近全微調                                │
└─────────────────────────────────────────────────────┘
```

In [ ]:
class LoRALayer(nn.Module):
    """LoRA 層：用於 Diffusion Model"""
    
    def __init__(
        self,
        original_layer: nn.Linear,
        rank: int = 4,
        alpha: float = 1.0,
        dropout: float = 0.0
    ):
        super().__init__()
        self.original_layer = original_layer
        self.rank = rank
        self.alpha = alpha
        self.scaling = alpha / rank
        
        in_features = original_layer.in_features
        out_features = original_layer.out_features
        
        # 凍結原始層
        for param in original_layer.parameters():
            param.requires_grad = False
            
        # LoRA 矩陣
        self.lora_A = nn.Linear(in_features, rank, bias=False)
        self.lora_B = nn.Linear(rank, out_features, bias=False)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        
        # 初始化
        nn.init.kaiming_uniform_(self.lora_A.weight, a=math.sqrt(5))
        nn.init.zeros_(self.lora_B.weight)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 原始輸出 + LoRA 調整
        original_output = self.original_layer(x)
        lora_output = self.lora_B(self.lora_A(self.dropout(x)))
        return original_output + self.scaling * lora_output
    
    def merge_weights(self):
        """合併 LoRA 權重到原始層（推理時使用）"""
        with torch.no_grad():
            # W' = W + scaling * B @ A
            delta_w = self.scaling * (self.lora_B.weight @ self.lora_A.weight)
            self.original_layer.weight.add_(delta_w)
            
    def get_lora_state_dict(self) -> Dict[str, torch.Tensor]:
        """獲取 LoRA 權重用於儲存"""
        return {
            'lora_A': self.lora_A.weight.data,
            'lora_B': self.lora_B.weight.data,
            'alpha': self.alpha,
            'rank': self.rank
        }


# 示範
original = nn.Linear(768, 768)
lora_layer = LoRALayer(original, rank=8, alpha=16)

print(f"原始層參數: {768 * 768:,} = {768 * 768 * 4 / 1024 / 1024:.2f} MB")
lora_params = sum(p.numel() for p in [lora_layer.lora_A.weight, lora_layer.lora_B.weight])
print(f"LoRA 參數: {lora_params:,} = {lora_params * 4 / 1024:.2f} KB")
print(f"參數減少: {(1 - lora_params / (768 * 768)) * 100:.2f}%")

In [ ]:
class LoRAInjector:
    """將 LoRA 注入到模型的指定層"""
    
    def __init__(
        self,
        model: nn.Module,
        target_modules: List[str] = ['to_q', 'to_k', 'to_v', 'to_out'],
        rank: int = 8,
        alpha: float = 16,
        dropout: float = 0.0
    ):
        self.model = model
        self.target_modules = target_modules
        self.rank = rank
        self.alpha = alpha
        self.dropout = dropout
        self.lora_layers = {}
        
    def inject(self) -> int:
        """
        注入 LoRA 層
        
        Returns:
            注入的層數
        """
        count = 0
        
        for name, module in self.model.named_modules():
            # 檢查是否是目標模組
            if any(target in name for target in self.target_modules):
                if isinstance(module, nn.Linear):
                    # 創建 LoRA 層
                    lora_layer = LoRALayer(
                        module, 
                        rank=self.rank,
                        alpha=self.alpha,
                        dropout=self.dropout
                    )
                    
                    # 替換原始層
                    parent_name = '.'.join(name.split('.')[:-1])
                    child_name = name.split('.')[-1]
                    
                    if parent_name:
                        parent = dict(self.model.named_modules())[parent_name]
                        setattr(parent, child_name, lora_layer)
                    
                    self.lora_layers[name] = lora_layer
                    count += 1
                    
        return count
    
    def get_trainable_params(self) -> List[nn.Parameter]:
        """獲取所有可訓練的 LoRA 參數"""
        params = []
        for lora_layer in self.lora_layers.values():
            params.extend([
                lora_layer.lora_A.weight,
                lora_layer.lora_B.weight
            ])
        return params
    
    def save_lora(self, path: str):
        """儲存 LoRA 權重"""
        state_dict = {}
        for name, layer in self.lora_layers.items():
            state_dict[name] = layer.get_lora_state_dict()
        torch.save(state_dict, path)
        print(f"LoRA 權重已儲存至 {path}")


# 示範用的簡單模型
class SimpleAttention(nn.Module):
    def __init__(self, dim=768):
        super().__init__()
        self.to_q = nn.Linear(dim, dim)
        self.to_k = nn.Linear(dim, dim)
        self.to_v = nn.Linear(dim, dim)
        self.to_out = nn.Linear(dim, dim)
        
    def forward(self, x):
        q = self.to_q(x)
        k = self.to_k(x)
        v = self.to_v(x)
        attn = F.softmax(q @ k.transpose(-2, -1) / math.sqrt(q.shape[-1]), dim=-1)
        return self.to_out(attn @ v)


# 測試注入
model = SimpleAttention()
injector = LoRAInjector(model, rank=8, alpha=16)
num_injected = injector.inject()

print(f"注入了 {num_injected} 個 LoRA 層")
print(f"可訓練參數: {len(injector.get_trainable_params())}")

## Part 4: ControlNet

ControlNet 為 Diffusion Model 添加額外的空間控制條件（邊緣、深度、姿態等）。

```
ControlNet 架構：

                     ┌──────────────────┐
                     │   Control Image  │
                     │  (edge/depth/...)│
                     └────────┬─────────┘
                              │
                              ▼
                     ┌──────────────────┐
                     │  Zero Convolution │ (訓練開始時輸出為 0)
                     └────────┬─────────┘
                              │
┌─────────────────────────────┼─────────────────────────────┐
│                             │                             │
│  ┌─────────────┐            │            ┌─────────────┐  │
│  │   Original  │            │            │  ControlNet │  │
│  │    UNet     │◄───────────┴───────────►│   (Copy)    │  │
│  │  (Locked)   │          加法            │(Trainable) │  │
│  └─────────────┘                          └─────────────┘  │
│                                                            │
└────────────────────────────────────────────────────────────┘

Zero Convolution 的作用：
- 訓練開始時，ControlNet 輸出為 0
- 模型行為與原始 SD 完全相同
- 隨著訓練進行，逐漸學習控制信號
```

In [ ]:
class ZeroConvolution(nn.Module):
    """Zero Convolution：初始化為零的卷積層"""
    
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
        # 初始化為零
        nn.init.zeros_(self.conv.weight)
        nn.init.zeros_(self.conv.bias)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv(x)


class ControlNetBlock(nn.Module):
    """ControlNet 的基本區塊"""
    
    def __init__(self, channels: int):
        super().__init__()
        
        # 控制編碼器
        self.control_encoder = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.SiLU(),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.SiLU()
        )
        
        # Zero convolution 輸出
        self.zero_conv = ZeroConvolution(channels, channels)
        
    def forward(self, x: torch.Tensor, control: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: UNet 特徵
            control: 控制信號
        """
        # 編碼控制信號
        control_features = self.control_encoder(control)
        
        # 通過 zero conv 後加到原始特徵
        return x + self.zero_conv(control_features)


class SimpleControlNet(nn.Module):
    """簡化版 ControlNet 示意"""
    
    def __init__(
        self,
        in_channels: int = 3,  # 控制圖像通道
        base_channels: int = 64,
        num_blocks: int = 4
    ):
        super().__init__()
        
        # 控制圖像編碼器
        self.input_hint_block = nn.Sequential(
            nn.Conv2d(in_channels, base_channels, 3, padding=1),
            nn.SiLU(),
            nn.Conv2d(base_channels, base_channels, 3, padding=1),
            nn.SiLU()
        )
        
        # Zero convolution for input
        self.input_zero_conv = ZeroConvolution(base_channels, base_channels)
        
        # ControlNet blocks
        self.blocks = nn.ModuleList([
            ControlNetBlock(base_channels * (2 ** min(i, 3)))
            for i in range(num_blocks)
        ])
        
        # Output zero convolutions
        self.output_zero_convs = nn.ModuleList([
            ZeroConvolution(base_channels * (2 ** min(i, 3)), 
                          base_channels * (2 ** min(i, 3)))
            for i in range(num_blocks)
        ])
        
    def forward(
        self,
        control_image: torch.Tensor,
        unet_features: List[torch.Tensor]
    ) -> List[torch.Tensor]:
        """
        Args:
            control_image: 控制圖像 [B, 3, H, W]
            unet_features: UNet 各層的特徵
            
        Returns:
            要加到 UNet 的控制信號
        """
        # 編碼控制圖像
        hint = self.input_hint_block(control_image)
        hint = self.input_zero_conv(hint)
        
        # 對每個 UNet 層產生控制信號
        control_outputs = []
        
        for block, zero_conv, unet_feat in zip(
            self.blocks, self.output_zero_convs, unet_features
        ):
            # 調整 hint 大小以匹配 UNet 特徵
            if hint.shape[-2:] != unet_feat.shape[-2:]:
                hint = F.interpolate(hint, size=unet_feat.shape[-2:], mode='bilinear')
                # 調整通道數
                if hint.shape[1] != unet_feat.shape[1]:
                    hint = F.conv2d(
                        hint, 
                        torch.randn(unet_feat.shape[1], hint.shape[1], 1, 1, device=hint.device) * 0.02
                    )
            
            control_out = zero_conv(hint)
            control_outputs.append(control_out)
            
        return control_outputs


# 測試
controlnet = SimpleControlNet()
print(f"ControlNet 參數量: {sum(p.numel() for p in controlnet.parameters()):,}")

# 驗證 zero convolution 初始輸出為 0
test_input = torch.randn(1, 64, 32, 32)
zero_conv = ZeroConvolution(64, 64)
print(f"Zero conv 輸出範圍: [{zero_conv(test_input).min():.6f}, {zero_conv(test_input).max():.6f}]")

In [ ]:
def visualize_control_types():
    """視覺化不同類型的控制條件"""
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    np.random.seed(42)
    size = 64
    
    # 第一行：控制圖像
    control_types = ['Canny Edge', 'Depth', 'Pose', 'Segmentation']
    
    # Canny Edge
    edge = np.zeros((size, size))
    edge[20:25, 10:50] = 1
    edge[20:50, 10:15] = 1
    edge[20:50, 45:50] = 1
    edge[45:50, 10:50] = 1
    axes[0, 0].imshow(edge, cmap='gray')
    axes[0, 0].set_title('Canny Edge')
    
    # Depth
    depth = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            depth[i, j] = 1 - abs(i - size//2) / size - abs(j - size//2) / size
    axes[0, 1].imshow(depth, cmap='viridis')
    axes[0, 1].set_title('Depth Map')
    
    # Pose (簡化的骨架)
    pose = np.zeros((size, size, 3))
    # 頭
    pose[10:15, 30:35] = [1, 0, 0]
    # 身體
    pose[15:35, 31:34] = [0, 1, 0]
    # 手臂
    pose[18:22, 20:32] = [0, 0, 1]
    pose[18:22, 33:45] = [0, 0, 1]
    # 腿
    pose[35:55, 28:31] = [1, 1, 0]
    pose[35:55, 34:37] = [1, 1, 0]
    axes[0, 2].imshow(pose)
    axes[0, 2].set_title('Pose')
    
    # Segmentation
    seg = np.zeros((size, size, 3))
    seg[:20, :] = [0.5, 0.8, 1.0]  # 天空
    seg[20:40, :] = [0.2, 0.6, 0.2]  # 樹
    seg[40:, :] = [0.6, 0.4, 0.2]  # 地面
    axes[0, 3].imshow(seg)
    axes[0, 3].set_title('Segmentation')
    
    # 第二行：對應的生成結果（模擬）
    results = [
        'Generated\nfrom Edge',
        'Generated\nfrom Depth',
        'Generated\nfrom Pose',
        'Generated\nfrom Seg'
    ]
    
    for i, (ax, title) in enumerate(zip(axes[1], results)):
        # 模擬生成結果
        img = np.random.rand(size, size, 3) * 0.3 + 0.4
        ax.imshow(img)
        ax.set_title(title)
    
    for ax in axes.flat:
        ax.axis('off')
        
    plt.suptitle('ControlNet: 不同控制條件類型', fontsize=14)
    plt.tight_layout()
    plt.show()

visualize_control_types()

## Part 5: 實際使用範例（使用 diffusers）

以下是使用 Hugging Face diffusers 庫進行客製化的程式碼範例。

In [ ]:
# Textual Inversion 訓練範例（使用 diffusers）
textual_inversion_code = '''
from diffusers import StableDiffusionPipeline
from diffusers.loaders import TextualInversionLoaderMixin

# 載入基礎模型
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# 訓練 Textual Inversion
# 使用 accelerate 命令：
# accelerate launch diffusers/examples/textual_inversion/textual_inversion.py \\
#   --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \\
#   --train_data_dir="./my_concept" \\
#   --learnable_property="object" \\
#   --placeholder_token="<my-concept>" \\
#   --initializer_token="dog" \\
#   --resolution=512 \\
#   --train_batch_size=1 \\
#   --gradient_accumulation_steps=4 \\
#   --max_train_steps=3000 \\
#   --learning_rate=5.0e-04 \\
#   --output_dir="./textual_inversion_output"

# 載入訓練好的嵌入
pipe.load_textual_inversion("./textual_inversion_output/learned_embeds.bin")

# 使用新概念生成
image = pipe("A <my-concept> in a forest").images[0]
'''
print("=== Textual Inversion 使用範例 ===")
print(textual_inversion_code)

In [ ]:
# DreamBooth 訓練範例
dreambooth_code = '''
# DreamBooth 訓練命令（使用 diffusers）
# accelerate launch diffusers/examples/dreambooth/train_dreambooth.py \\
#   --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \\
#   --instance_data_dir="./my_dog" \\
#   --output_dir="./dreambooth_output" \\
#   --instance_prompt="a photo of sks dog" \\
#   --resolution=512 \\
#   --train_batch_size=1 \\
#   --gradient_accumulation_steps=1 \\
#   --learning_rate=5e-6 \\
#   --lr_scheduler="constant" \\
#   --lr_warmup_steps=0 \\
#   --max_train_steps=800 \\
#   --with_prior_preservation \\
#   --prior_loss_weight=1.0 \\
#   --class_data_dir="./class_dog" \\
#   --class_prompt="a photo of dog" \\
#   --num_class_images=200

# 使用訓練好的模型
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "./dreambooth_output",
    torch_dtype=torch.float16
).to("cuda")

image = pipe("a photo of sks dog on the beach").images[0]
'''
print("=== DreamBooth 使用範例 ===")
print(dreambooth_code)

In [ ]:
# LoRA 訓練與使用範例
lora_code = '''
# LoRA 訓練（使用 diffusers + PEFT）
# accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora.py \\
#   --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \\
#   --instance_data_dir="./my_style" \\
#   --output_dir="./lora_output" \\
#   --instance_prompt="artwork in style of sks" \\
#   --resolution=512 \\
#   --train_batch_size=1 \\
#   --gradient_accumulation_steps=4 \\
#   --learning_rate=1e-4 \\
#   --max_train_steps=1000 \\
#   --rank=8

# 使用 LoRA
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# 載入 LoRA 權重
pipe.load_lora_weights("./lora_output")

# 調整 LoRA 強度
pipe.fuse_lora(lora_scale=0.8)  # 0-1 之間

image = pipe("a cat in style of sks").images[0]

# 移除 LoRA（恢復原始模型）
pipe.unfuse_lora()
pipe.unload_lora_weights()
'''
print("=== LoRA 使用範例 ===")
print(lora_code)

In [ ]:
# ControlNet 使用範例
controlnet_code = '''
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers.utils import load_image
import cv2
import numpy as np

# 載入 ControlNet（Canny 邊緣檢測版本）
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to("cuda")

# 準備控制圖像
image = load_image("your_image.png")
image = np.array(image)

# Canny 邊緣檢測
low_threshold = 100
high_threshold = 200
canny_image = cv2.Canny(image, low_threshold, high_threshold)
canny_image = canny_image[:, :, None]
canny_image = np.concatenate([canny_image, canny_image, canny_image], axis=2)

# 生成
output = pipe(
    "a beautiful landscape",
    image=canny_image,
    num_inference_steps=30,
).images[0]

# 多個 ControlNet 組合
from diffusers import MultiControlNetModel

controlnet_canny = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)
controlnet_depth = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-depth", torch_dtype=torch.float16
)

multi_controlnet = MultiControlNetModel([controlnet_canny, controlnet_depth])

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=multi_controlnet,
    torch_dtype=torch.float16
).to("cuda")

output = pipe(
    "a beautiful scene",
    image=[canny_image, depth_image],
    controlnet_conditioning_scale=[0.5, 0.5],  # 調整各自的影響力
).images[0]
'''
print("=== ControlNet 使用範例 ===")
print(controlnet_code)

## Part 6: 方法比較與選擇指南

In [ ]:
def print_comparison_table():
    """印出客製化方法比較表"""
    print("""
╔══════════════════════════════════════════════════════════════════════════════════╗
║                     Diffusion Model 客製化方法比較                               ║
╠═══════════════════╦═══════════════╦═══════════════╦═══════════════╦══════════════╣
║       方法        ║ Textual Inv.  ║  DreamBooth   ║     LoRA      ║  ControlNet  ║
╠═══════════════════╬═══════════════╬═══════════════╬═══════════════╬══════════════╣
║ 訓練參數          ║    ~768       ║    ~1B        ║    ~4M        ║   ~361M      ║
║ VRAM 需求         ║    ~8GB       ║   ~24GB       ║   ~12GB       ║   ~16GB      ║
║ 訓練時間          ║   1-2小時     ║   30分鐘      ║   1小時       ║   數天       ║
║ 儲存大小          ║    ~4KB       ║   ~4GB        ║   ~50MB       ║   ~1.5GB     ║
╠═══════════════════╬═══════════════╬═══════════════╬═══════════════╬══════════════╣
║ 適用場景          ║ 學習新概念    ║ 特定主題      ║ 風格/概念     ║ 空間控制     ║
║                   ║ 風格遷移      ║ 人物/物品     ║ 靈活組合      ║ 姿態/邊緣    ║
╠═══════════════════╬═══════════════╬═══════════════╬═══════════════╬══════════════╣
║ 優點              ║ 極小儲存      ║ 效果最好      ║ 可組合        ║ 精確控制     ║
║                   ║ 可組合        ║ 保真度高      ║ 效率高        ║ 保持結構     ║
╠═══════════════════╬═══════════════╬═══════════════╬═══════════════╬══════════════╣
║ 缺點              ║ 表達力有限    ║ 需要大VRAM    ║ 效果略遜      ║ 需要控制圖   ║
║                   ║               ║ 不可組合      ║               ║ 訓練成本高   ║
╠═══════════════════╬═══════════════╬═══════════════╬═══════════════╬══════════════╣
║ 所需圖片          ║    3-5張      ║    3-5張      ║   10-50張     ║   大量成對   ║
╚═══════════════════╩═══════════════╩═══════════════╩═══════════════╩══════════════╝

選擇指南：
┌────────────────────────────────────────────────────────────────────┐
│ 需求                                    │ 推薦方法               │
├────────────────────────────────────────────────────────────────────┤
│ 學習一個新的視覺概念（如特定物品）      │ Textual Inversion      │
│ 生成特定人物/寵物的高品質圖片           │ DreamBooth             │
│ 學習特定藝術風格並靈活應用              │ LoRA                   │
│ 組合多個客製化概念                      │ Textual Inv. + LoRA    │
│ 需要精確的空間/姿態控制                 │ ControlNet             │
│ 有限的 GPU 記憶體                       │ Textual Inv. 或 LoRA   │
│ 需要最佳品質，不在意資源                │ DreamBooth             │
└────────────────────────────────────────────────────────────────────┘
""")

print_comparison_table()

## 練習

### Exercise 1: 實作 LoRA 權重合併

實作一個函數，能夠將多個 LoRA 適配器合併到一起。

In [ ]:
def merge_lora_weights(
    lora_weights_list: List[Dict[str, torch.Tensor]],
    weights: List[float]
) -> Dict[str, torch.Tensor]:
    """
    合併多個 LoRA 權重
    
    Args:
        lora_weights_list: LoRA 權重字典列表
        weights: 各個 LoRA 的權重
        
    Returns:
        合併後的 LoRA 權重
    """
    # TODO: 實作權重合併
    # 提示：
    # 1. 遍歷所有 LoRA 權重
    # 2. 對相同 key 的權重進行加權平均
    # 3. 處理可能的維度不匹配
    pass

# 測試
# lora1 = {'layer1.lora_A': torch.randn(8, 768), 'layer1.lora_B': torch.randn(768, 8)}
# lora2 = {'layer1.lora_A': torch.randn(8, 768), 'layer1.lora_B': torch.randn(768, 8)}
# merged = merge_lora_weights([lora1, lora2], [0.5, 0.5])

### Exercise 2: 實作控制條件預處理

實作不同類型控制條件的預處理函數。

In [ ]:
class ControlImageProcessor:
    """控制圖像預處理器"""
    
    @staticmethod
    def to_canny(
        image: np.ndarray,
        low_threshold: int = 100,
        high_threshold: int = 200
    ) -> np.ndarray:
        """
        Canny 邊緣檢測
        
        Args:
            image: RGB 圖像 [H, W, 3]
            low_threshold: 低閾值
            high_threshold: 高閾值
            
        Returns:
            邊緣圖像 [H, W, 3]
        """
        # TODO: 實作 Canny 邊緣檢測
        # 提示：使用 cv2.Canny 或手動實作
        pass
    
    @staticmethod
    def to_depth(image: np.ndarray) -> np.ndarray:
        """
        估計深度圖（簡化版本）
        
        Args:
            image: RGB 圖像
            
        Returns:
            深度圖 [H, W, 3]
        """
        # TODO: 實作深度估計
        # 提示：可以使用亮度作為簡化的深度估計
        pass

# 測試
# processor = ControlImageProcessor()
# test_image = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
# canny = processor.to_canny(test_image)

### Exercise 3: 設計訓練策略

根據不同的使用場景，設計合適的客製化訓練策略。

In [ ]:
@dataclass
class CustomizationStrategy:
    """客製化策略配置"""
    method: str  # 'textual_inversion', 'dreambooth', 'lora'
    learning_rate: float
    train_steps: int
    batch_size: int
    use_prior_preservation: bool
    lora_rank: Optional[int] = None
    num_vectors: Optional[int] = None  # for textual inversion
    notes: str = ""


def recommend_strategy(
    use_case: str,
    num_images: int,
    available_vram_gb: int
) -> CustomizationStrategy:
    """
    根據使用場景推薦客製化策略
    
    Args:
        use_case: 使用場景
            - 'style': 學習藝術風格
            - 'person': 學習特定人物
            - 'object': 學習特定物品
            - 'concept': 學習抽象概念
        num_images: 可用的訓練圖片數量
        available_vram_gb: 可用 VRAM (GB)
        
    Returns:
        推薦的策略配置
    """
    # TODO: 實作策略推薦邏輯
    # 考慮因素：
    # 1. VRAM 限制
    # 2. 圖片數量
    # 3. 使用場景的特性
    pass

# 測試
# strategy = recommend_strategy('style', num_images=10, available_vram_gb=16)
# print(strategy)

## 總結

```
Diffusion 客製化重點回顧：

1. Textual Inversion
   ├─ 只學習新的文字嵌入
   ├─ 參數極少（~768）
   ├─ 適合學習新概念
   └─ 可與其他方法組合

2. DreamBooth
   ├─ 微調整個 UNet
   ├─ 需要 prior preservation
   ├─ 效果最好但資源需求高
   └─ 適合學習特定主題

3. LoRA
   ├─ 低秩分解，高效訓練
   ├─ 可以組合多個 LoRA
   ├─ 儲存空間小（~50MB）
   └─ 平衡效果與效率

4. ControlNet
   ├─ 添加空間控制條件
   ├─ 使用 zero convolution
   ├─ 支援多種控制類型
   └─ 適合需要精確控制的場景

實際應用建議：
- RTX 5080 (16GB): 可運行 LoRA、Textual Inversion
- DreamBooth 需要 gradient checkpointing 或降低解析度
- 優先嘗試 LoRA，效果不佳再考慮 DreamBooth
```